In [467]:
import string
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer

STOPWORDS = set(stopwords.words('english')) 
URL_STOP_WORDS = set(["http", "https", "www", "ftp", "com", "net", "org", "archives", "pdf", "html", "png", "txt", "redirect", "align", "realign", "valign", "nonalign", "malign", "unalign", "salign", "qalign", "halign", "font", "fontsiz", "fontcolor", "backgroundcolor", "background", "style", "center", "text"])


In [468]:
def readIndex():
    f= open("InvertedIndex.txt", "r")
    Index= {}

    lines= f.readlines()
    line =0

    while line< len(lines):
        text = lines[line].split(":")
        word= text[0]
        frequency= text[1]
        line +=1
        while line< len(lines) and ":" not in lines[line]:
            frequency += lines[line]
            line +=1
        Index[word]= frequency

    f.close()
    return Index

In [469]:
def cleanQuery(query):
    stemmer= SnowballStemmer("english")
    query = re.sub(r'\\d+', '', query) #Remove numbers
    query = re.sub(r'<(.*?)>','',query) #Remove tags if any
    query = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', query, flags=re.MULTILINE) #Remove Url
    query = re.sub(r'{\|(.*?)\|}', '', query, flags=re.MULTILINE) #Remove CSS
    query = re.sub(r'\[\[file:(.*?)\]\]', '', query, flags=re.MULTILINE) #Remove File
    query = re.sub(r'[^\w\s]', '', query) 
    query = query.split()
    query = [x for x in query if x not in STOPWORDS and x not in URL_STOP_WORDS]
    query = [stemmer.stem(word) for word in query]
#     print ("Query in Processing: {}".format(query))
    
    return query

In [470]:
def search_fieldQuery(documents, field_type):
    fieldDocs= []
    documents= documents.split("|")
    for doc in documents:
        if field_type in doc:
            fieldDocs.append(doc)
    return fieldDocs

In [471]:
def process_fieldQuery(query):
    fieldInfo= {}
    
    field= query.find("t:")
    if field !=-1:
        fieldInfo[field]= "t:"
        
    field= query.find("i:")
    if field !=-1:
        fieldInfo[field]= "i:"
        
    field= query.find("c:")
    if field !=-1:
        fieldInfo[field]= "c:"
        
    field= query.find("b:")
    if field !=-1:
        fieldInfo[field]= "b:"
    
    field= query.find("r:")
    if field !=-1:
        fieldInfo[field]= "r:"
        
    field= query.find("l:")
    if field !=-1:
        fieldInfo[field]= "l:"
    
    fieldInfo= sorted(fieldInfo.items())
    fieldInfo.append((1234567890, ""))
#     print(fieldInfo)
    i=0
    while i+1 <len(fieldInfo):
        fieldQuery = (query[fieldInfo[i][0]+2 : fieldInfo[i+1][0]]).lower()
#         print (fieldQuery)
        fieldQuery = cleanQuery(fieldQuery)
#         print (fieldQuery)
        for word in fieldQuery:
            if word not in Index:
                print (word, " : ", [])
            else:
                value= Index[word]
                print(word, " : ", search_fieldQuery(value, fieldInfo[i][1][:1]))
            print()
        i +=1
    

In [472]:
Index= readIndex()

query= "Sachin%$TWEGSDGEY&^UHG GFEYRYGERG$^TRE Ramesh Tendlkar"

if "t:" in query or "i:" in query or "c:" in query or "b:"in query or "r:" in query or "l:" in query:    
    process_fieldQuery(query)
else:
    query = cleanQuery(query)
    for word in query:
        if word.lower() in Index:
            print (word, " : ", Index[word.lower()])
        else:
            print (word, " : ", [])

sachintwegsdgeyuhg  :  []
gfeyrygergtr  :  []
ramesh  :  185-i2|588-b1|1462-b1|1763-b13|2173-b1|2718-b1|3081-i3|3159-b1|3602-b1|4133-b1|4175-b2|4434-b1|4577-b1|5013-b1|5137-b1|5634-b1|6039-b1|6875-b1|7396-b1|8296-b1|8553-i2|8616-b1|8777-b1|8831-b3|8993-b1|9290-b1|10097-b1|10209-b1|10260-b1|10799-b3|11186-b1|11705-b1|11721-b1|11997-b1|12184-b1|12347-b1|12406-b1|12583-b1|12625-b1|12725-b1|12864-b1|13003-b1|13217-b1|13544-b1|13849-b1|13950-b1|14210-b1|14224-b1|14386-b1|15201-b1|15331-b1|16099-b1|16219-b1|16363-b1|16651-b1|16848-b1|17080-b1|17706-b1|17904-b2|18158-b2|18193-b1|19211-b1|19508-b2|19623-b1

tendlkar  :  []
